In [4]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

In [5]:
## Define o sentimento e o idioma da mensagem enviada

class Sentimento(BaseModel):
    sentimento: str = Field(description="Sentimento atribuído ao texto. Deve ser 'pos' para positivo, 'neg' para negativo e 'nt' que significa neutro quando não identificado")
    lingua: str = Field(description="Em qual idioma a mensagem foi enviada (deve estar no charset UTF-8)")
    
tool_get_sentimento = convert_to_openai_function(Sentimento)

In [12]:
text = "I hate my college, it's the worst envioroment that i know "

In [17]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente especializado em análise de sentimentos e detecção de idioma.",
        ),
        (
            "user",
            "Receba o texto abaixo e identifique:\n"
            "1. O sentimento ('pos' para positivo, 'neg' para negativo e 'nt' para neutro).\n"
            "2. O idioma em que o texto está escrito.\n\n"
            "Texto: {input}",
        ),
    ]
)

chain = prompt | chat.bind(functions = [tool_get_sentimento], function_call = {"name": "Sentimento"})

response = chain.invoke({"input": text})

print(response)

content='' additional_kwargs={'function_call': {'arguments': '{"sentimento":"neg","lingua":"en"}', 'name': 'Sentimento'}} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 179, 'total_tokens': 190, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'stop', 'logprobs': None} id='run-4d0aadf6-e5be-4c2e-93dc-a357686f358f-0'


### Outro metodo de pegar tagging - Classificando setores dentro de uma empresa

In [47]:
# Lista de solicitações
solicitacoes = [
    # SUPORTE_TECNICO
    "Meu computador está travando constantemente ao abrir programas pesados como editores de vídeo. Preciso de ajuda para resolver isso.",
    "Durante a instalação do software de design gráfico, recebo uma mensagem de erro indicando falta de permissões. Como posso corrigir isso?",
    "O aplicativo de armazenamento em nuvem não está sincronizando meus arquivos corretamente. Já tentei reinstalar, mas o problema persiste.",
    "A impressora da empresa não está sendo reconhecida pela rede local. Já reiniciei o roteador e a impressora, mas continua fora da rede.",
    "Meu antivírus expirou e agora o computador está apresentando lentidão extrema. Preciso de ajuda para atualizar ou reinstalar a proteção.",
    # VENDAS
    "Gostaria de receber uma proposta comercial personalizada para renovar o contrato anual da nossa empresa.",
    "Poderiam me enviar o catálogo atualizado de produtos e os preços para revenda?",
    "Quero saber quais são as condições de pagamento e se há descontos para compras em grande volume.",
    "Estou interessado em adquirir licenças adicionais do seu software. Como funciona o processo de compra?",
    "Vocês oferecem algum plano corporativo que atenda empresas de médio porte?",
    # MARKETING
    "Gostaria de discutir uma possível parceria para divulgação de nossa marca nas suas plataformas.",
    "Estamos organizando um evento e queremos saber como contratar o patrocínio de vocês.",
    "Quais são as opções de publicidade disponíveis para empresas dentro do seu aplicativo?",
    "Gostaríamos de obter dados demográficos da sua base de clientes para avaliar uma ação de co-marketing.",
    "Estou interessado em publicar um artigo patrocinado no blog da empresa. Quais são as condições?",
]

In [45]:
from enum import Enum

class Setor(Enum):
    SUPORTE_TECNICO = "Suporte Técnico"
    VENDAS = "Vendas"
    MARKETING = "Marketing"


class SetorEnum(str, Enum):
    SUPORTE_TECNICO = "Suporte Técnico"
    VENDAS = "Vendas"
    MARKETING = "Marketing"

class Tag(BaseModel):
    setor: SetorEnum = Field(description="O setor ao qual a solicitação pertence")
    prioridade: str = Field(description="A prioridade da solicitação, deve ser 'alta', 'média' ou 'baixa'")





In [48]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_tool

from enum import Enum


class Setor(Enum):
    SUPORTE_TECNICO = "Suporte Técnico"
    VENDAS = "Vendas"
    MARKETING = "Marketing"


class SetorEnum(str, Enum):
    SUPORTE_TECNICO = "Suporte Técnico"
    VENDAS = "Vendas"
    MARKETING = "Marketing"


class Tag(BaseModel):
    setor: SetorEnum = Field(description="O setor ao qual a solicitação pertence")
    prioridade: str = Field(
        description="A prioridade da solicitação, deve ser 'alta', 'média' ou 'baixa'"
    )

tool_get_tag = convert_pydantic_to_openai_tool(Tag)


from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_tools import JsonOutputToolsParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente especializado em classificar solicitações de suporte técnico."
            "\n\nAs categorias possíveis são:"
            "\n- SUPORTE_TECNICO: Solicitações relacionadas a problemas técnicos, como dificuldades com hardware, software, configuração de sistemas ou dúvidas técnicas em geral."
            "\n  Exemplo: 'Meu computador não liga', 'Não consigo acessar meu e-mail'."
            "\n\n- VENDAS: Solicitações relacionadas à compra de produtos ou serviços, informações sobre preços, planos, propostas comerciais, ou renovação de contratos."
            "\n  Exemplo: 'Gostaria de saber os preços dos seus planos', 'Como faço para renovar minha assinatura?'."
            "\n\n- MARKETING: Solicitações relacionadas a divulgação, campanhas publicitárias, eventos, parcerias de marketing ou estratégias de branding."
            "\n  Exemplo: 'Quero saber mais sobre parceria para eventos', 'Gostaria de informações sobre oportunidades de divulgação em sua plataforma'."
            "\n\nClassifique a solicitação recebida de acordo com o conteúdo e a intenção do solicitante. "
            "Responda apenas com o nome da categoria adequada: SUPORTE_TECNICO, VENDAS ou MARKETING.",
        ),
        (
            "user",
            "Texto da solicitação:\n"
            "{solicitacao}\n\n"
            "Responda apenas com o nome da categoria adequada.",
        ),
    ]
)

chain = (
    prompt
    | chat.bind(
        tools=[tool_get_tag],  # ✅ Aqui: tools e não functions
        tool_choice={
            "type": "function",
            "function": {"name": "Tag"},
        },  # ✅ Garante que vai usar a função Tag
    )
    | JsonOutputToolsParser()
)


solicitacao = solicitacoes[7]

resposta = chain.invoke({"solicitacao": solicitacao})

print(solicitacao)
print(resposta)

Quero saber quais são as condições de pagamento e se há descontos para compras em grande volume.
[{'args': {'setor': 'Vendas', 'prioridade': 'média'}, 'type': 'Tag'}]
